In [ ]:
!pip install transformers accelerate einops langchain langchain-community \
bitsandbytes
sentence_transformers
langchain-chroma
rank_bm25

In [ ]:
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

## Download data

In [7]:
# download unparsed docs
# !gdown 1-bIiFVRemNVcW8cFWgj_sc-s30_wIh-d
# !unzip otraslevye_docs_gost.

# # download parsed docs
# !gdown 1g_EjrREtsZEBaXqvcpseTDxAra4eG7xy
# !unzip -q otraslevye_docs_gost_chunks.zip

# # download embeddings db
# !gdown 1WuhcGHRO77d3vkI0NEw-ElMxQOmI5Up0
# !unzip -q chunks_chroma_db.zip

# download embeddings db with ids
!gdown 1iam7mh_zT_7Kh29YDvN2SZIQJkvleDiC
!unzip -q chroma_langchain_db.zip

# download parsed docs
!gdown 1-4HcInLPjs8F3Y5DCMDIKT8PhARMLU0b

# download test qa
!gdown 1AozSEFxL-f4T5alEnunuvh2BXOzMin8e

Downloading...
From (original): https://drive.google.com/uc?id=1iam7mh_zT_7Kh29YDvN2SZIQJkvleDiC
From (redirected): https://drive.google.com/uc?id=1iam7mh_zT_7Kh29YDvN2SZIQJkvleDiC&confirm=t&uuid=88e0d683-dd83-4e45-a4ab-72db61b2db3c
To: /home/davm/chroma_langchain_db.zip
100%|█████████████████████████████████████████| 119M/119M [00:01<00:00, 102MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-4HcInLPjs8F3Y5DCMDIKT8PhARMLU0b
To: /home/davm/otraslevye_docs_gost.json
100%|███████████████████████████████████████| 65.8M/65.8M [00:00<00:00, 101MB/s]
Downloading...
From: https://drive.google.com/uc?id=1AozSEFxL-f4T5alEnunuvh2BXOzMin8e
To: /home/davm/qa_on_gost.csv
100%|████████████████████████████████████████| 100k/100k [00:00<00:00, 13.1MB/s]


## Imports

In [1]:
import os
os.chdir('public/itmo-sai-rag')

In [2]:
import os
import json
import torch
import transformers
import pandas as pd
from time import time
from tqdm.notebook import tqdm
import chromadb
from langchain.llms import HuggingFacePipeline
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA
from unsloth import FastLanguageModel
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from rag_pipeline import RAGPipeline

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [3]:
from langchain.globals import set_verbose

set_verbose(True)

## Initialize LLM

In [4]:
max_seq_length = 2048  # Choose any! We auto support RoPE Scaling internally!
dtype = None  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True  # Use 4bit quantization to reduce memory usage. Can be False.

In [5]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

FastLanguageModel.for_inference(model);  # Enable native 2x faster inference

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.568 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu118. CUDA = 7.5. CUDA Toolkit = 11.8.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.22.post7+cu118. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [6]:
query_pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    max_length=2048,
    max_new_tokens=1024,
    device_map="auto", )

In [7]:
llm = HuggingFacePipeline(pipeline=query_pipeline)

/tmp/ipykernel_11222/266891025.py:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  llm = HuggingFacePipeline(pipeline=query_pipeline)


## Load documents

In [ ]:
with open('otraslevye_docs_gost.json') as f:
    chunks = json.load(f)

In [ ]:
documents = []
for i in range(len(chunks['documents'])):
    page_content = chunks['documents'][i]
    chapter = chunks['metadatas'][i]['chapter']
    file_name = chunks['metadatas'][i]['file_name']
    source = chunks['metadatas'][i]['source']
    metadata = {
        'chapter': chapter,
        'file_name': file_name,
        'source': source
    }
    documents.append(Document(page_content=page_content, metadata=metadata))

ids = chunks['ids']

In [ ]:
def load_chunks(file_path):
    with open(file_path) as f:
        data = json.load(f)

    docs = []
    for i in data:
        chunk = data[i]
        page_content = chunk['kwargs']['page_content']
        source = chunk['kwargs']['metadata']['source']
        metadata = {
            'source': source
        }
        docs.append(Document(page_content=page_content, metadata=metadata))

    return docs

In [ ]:
chunks_dir = '/content/otraslevye_docs_gost_chunks'

In [ ]:
documents = []
for i in os.scandir(chunks_dir):
    documents.extend(load_chunks(i.path))

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
all_splits = text_splitter.split_documents(documents)

In [ ]:
with open('otraslevye_docs_gost.json') as f:
    chunks = json.load(f)
chunks.pop('data')
chunks = pd.DataFrame(chunks)

In [ ]:
chunks

,ids,embeddings,metadatas,documents,uris
0,00040bdc-a7fc-4ab0-bac8-64a1097f7974,None,"{'chapter': '', 'file_name': 'ВСН 22-79 Времен...","4. СБОР, ПОДГОТОВКА И ТРАНСПОРТ НЕФТИ И ГАЗА С...",None
1,0011b29e-8845-4275-bd6c-f5a3f97b4605,None,"{'chapter': 'Промышленная, пожарная безопаснос...",18.1.5 Запрещается ввод в эксплуатацию и работ...,None
2,0018d92f-2780-42b5-b03c-36b2ee590b65,None,"{'chapter': 'РД 153-39.0-047-00', 'file_name':...","от ОАО ЦГЭ - д.т.н., акад. РАЕН Кашик А. С., д...",None
3,0040d17f-1b18-47b5-af36-bda114f769b2,None,{'chapter': '1 — предохранительный клапан; 2 -...,Схема отбора проб из трубопровода с применение...,None
4,00468383-11a5-44e5-a766-b8014110e1a4,None,"{'chapter': '4 Общие положения', 'file_name': ...",В ГОСТ 34233.1 — ГОСТ 34233.11 рассматриваются...,None
...,...,...,...,...,...
9389,ffea877a-0995-4b85-ba07-38669843b873,None,"{'chapter': 'поверочная жидкость: Жидкость, пр...",Примечания ' Б качества поверочной жидкости пр...,None
9390,ffebbfd0-1124-4e0c-938f-6ff93dd20ee8,None,{'chapter': 'Требования к поставщику/изготовит...,Проверка прочности на растяжение и прочности с...,None
9391,fff3af64-8548-4893-851f-a96a58702c6b,None,{'chapter': '5 Процессы и мероприятия по обесп...,Данный процесс охватывает фактическое проведен...,None
9392,fff6f5da-b002-4caf-bdf1-3edfb49f818c,None,{'chapter': 'АППАРАТЫ С ВОЗДУШНЫМ ОХЛАЖДЕНИЕМ'...,Общий технические требования Air-cooled appara...,None


## Create embeddings and database

In [8]:
model_path = 'intfloat/multilingual-e5-large'
model_name = model_path.split('/')[-1]
model_kwargs = {"device": "cuda"}

# try to access the sentence transformers from HuggingFace: https://huggingface.co/api/models/sentence-transformers/all-mpnet-base-v2
try:
    embeddings = HuggingFaceEmbeddings(model_name=model_path, model_kwargs=model_kwargs)
except Exception as ex:
    print("Exception: ", ex)
    # alternatively, we will access the embeddings models locally
    local_model_path = "/kaggle/input/sentence-transformers/minilm-l6-v2/all-MiniLM-L6-v2"
    print(f"Use alternative (local) model: {local_model_path}\n")
    embeddings = HuggingFaceEmbeddings(model_name=local_model_path, model_kwargs=model_kwargs)

# embedding_function = SentenceTransformerEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

/tmp/ipykernel_11222/550877843.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name=model_path, model_kwargs=model_kwargs)


In [9]:
chroma_path = f'db/chroma_langchain_db_{model_name}'

vectordb = Chroma(
    collection_name="otraslevye_docs_gost",
    embedding_function=embeddings,
    persist_directory=chroma_path
)

In [ ]:
for document, id in tqdm(zip(documents, ids), total=len(documents)):
    vectordb.add_documents(documents=[document], ids=[id])

In [ ]:
client = chromadb.PersistentClient()
collection = client.get_or_create_collection("otraslevye_docs_gost", embedding_function=embeddings)

In [ ]:
for chunk_row in tqdm(chunks.iterrows(), total=len(chunks)):
    collection.add(**dict(chunk_row[1]))
    break

  0%|          | 0/9394 [00:00<?, ?it/s]

In [ ]:
client = chromadb.PersistentClient()
collection = client.get_or_create_collection("otraslevye_docs_gost", embedding_function=embedding_function)
for chunk_row in tqdm(chunks.iterrows(), total=len(chunks)):
    collection.add(**dict(chunk_row[1]))

  0%|          | 0/9394 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
vectordb = Chroma(
    client=client,
    collection_name="otraslevye_docs_gost",
    embedding_function=embedding_function,
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [ ]:
vectordb = Chroma.from_documents(documents=documents, embedding=embeddings, persist_directory="chroma_db")

In [ ]:
vectordb = Chroma(persist_directory='chroma_db', embedding_function=embeddings)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [ ]:
!zip -r chroma_langchain_db.zip chroma_langchain_db

  adding: chroma_langchain_db/ (stored 0%)
  adding: chroma_langchain_db/e742eb1f-96ac-4320-8b98-c63b96c8b2ce/ (stored 0%)
  adding: chroma_langchain_db/e742eb1f-96ac-4320-8b98-c63b96c8b2ce/header.bin (deflated 54%)
  adding: chroma_langchain_db/e742eb1f-96ac-4320-8b98-c63b96c8b2ce/link_lists.bin (deflated 79%)
  adding: chroma_langchain_db/e742eb1f-96ac-4320-8b98-c63b96c8b2ce/length.bin (deflated 92%)
  adding: chroma_langchain_db/e742eb1f-96ac-4320-8b98-c63b96c8b2ce/index_metadata.pickle (deflated 44%)
  adding: chroma_langchain_db/e742eb1f-96ac-4320-8b98-c63b96c8b2ce/data_level0.bin (deflated 9%)
  adding: chroma_langchain_db/chroma.sqlite3 (deflated 59%)


In [10]:
vectordb.similarity_search(
    'Какова максимально допустимая разность между внутренним диаметром кожуха и наружным диаметром поперечных перегородок?')

[Document(metadata={'chapter': 'Сварка', 'file_name': 'ГОСТ 31842-2012 Нефтяная и газовая промышленность. Теплообменники кожухотрубчатые. Технические требования.docx', 'source': '/mnt/docs/ГОСТ 31842-2012 Нефтяная и газовая промышленность. Теплообменники кожухотрубчатые. Технические требования.docx'}, page_content='Продольное отклонение внутреннего диаметра кожуха конденсаторов и испарителей с паровым теплоносителем в можтрубном npocmpai temae. а также испаритслой с паровым пространством должно соответствовать Н16по ГОСТ 25347. 7.6.7 Предельное отклонение наружного диаметра поперечных перегородок должно соответствовать 61 Зло ГОСТ25347 7.6.3 Максимально допустимая разность между внутренним диаметром кожуха и наружным диаметром поперечных перегородок должна соответствовать значению, рассчитанному с учетом предельных отклонений внутреннего диаметра кожуха и наружного диаметра поперечных перегородок. указанных в 7.6 6 и 7.6.7. Диаметральный зазор между кожухом и поперечной перегородкой дл

## Initialize chain   


In [20]:
rag_pipeline = RAGPipeline(llm, vectordb, return_intermediate_results=True, hyde=False, do_planning=True)

In [21]:
rag_chain = rag_pipeline.build_chain()

In [12]:
rag_chain.invoke('Какова допустимая разность диаметрального зазора между кожухом и поперечной перегородкой для аппаратов типов Н и К, внутренний диаметр которых больше 1400 мм?')

Both `max_new_tokens` (=1024) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


{'response': '\n<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nОтветь на вопрос по нескольким параграфам. Для ответа используй только информацию в представленных параграфах.\nЕсли на вопрос нельзя ответить исходя из параграфов, напиши "недостаточно информации для ответа".\n<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nПараграф 1: <tr><td>5/                                            </td><td>1400                                                                   </td><td>14UU                                                                   </td><td>1400                                                                   </td><td>1400                                                                   </td></tr>\n</tbody>\n</table> Примечание— Максимальное расстояние нежду поперечными переасройлами для испарит алей с ларсеым пространствам должна соотвалять 12(!ймт. 6.1.10 Минимальное расстояние между поперечными перегородками может составлять 0.2 внутреннего диаметра

## Test RAG system

In [ ]:
from IPython.display import display, Markdown


def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [ ]:
def test_rag(qa, query):
    time_start = time()
    response = qa.invoke(input=query)
    time_end = time()
    total_time = f"{round(time_end - time_start, 3)} sec."

    full_response = f"Question: {query}\nAnswer: {response}\nTotal time: {total_time}"
    display(Markdown(colorize_text(full_response)))

In [ ]:
# russian prompt
query = "Какова допустимая разность диаметрального зазора между кожухом и поперечной перегородкой для аппаратов типов Н и К, внутренний диаметр которых больше 1400 мм?"
test_rag(qa, query)

Both `max_new_tokens` (=1024) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)




> Entering new RetrievalQA chain...

> Finished chain.




**<font color='red'>Question:</font>** Какова допустимая разность диаметрального зазора между кожухом и поперечной перегородкой для аппаратов типов Н и К, внутренний диаметр которых больше 1400 мм?


**<font color='green'>Answer:</font>** 
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
Ответь на вопрос по трем параграфам. Для ответа используй только информацию в представленных параграфах.
Если на вопрос нельзя ответить исходя из параграфов, напиши "недостаточно информации для ответа".
<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Продольное отклонение внутреннего диаметра кожуха конденсаторов и испарителей с паровым теплоносителем в можтрубном npocmpai temae. а также испаритслой с паровым пространством должно соответствовать Н16по ГОСТ 25347. 7.6.7 Предельное отклонение наружного диаметра поперечных перегородок должно соответствовать 61 Зло ГОСТ25347 7.6.3 Максимально допустимая разность между внутренним диаметром кожуха и наружным диаметром поперечных перегородок должна соответствовать значению, рассчитанному с учетом предельных отклонений внутреннего диаметра кожуха и наружного диаметра поперечных перегородок. указанных в 7.6 6 и 7.6.7. Диаметральный зазор между кожухом и поперечной перегородкой для аппаратов типов Н и К, внутренний диаметр которых долее 1400 мм. допускается не более 10 мм. 7.6.0 При изготовлении плавающей головки размер высоты .6 накладки 1, угол и расстояние между фланцем и полукольцом, а также допуски на наружный диаметр решетки D э, внупфюмной dua- мстр полукольца D4, диаметр вытечки полукольца (D, + 2) и угол должны соответствовать указанным на рисунке 6. 7.6.0.1 Предельное отклонение внутреннего диаметра полукольца должно соответствовать Н ? 1 по ГОСТ 25347 и быть обеспечено для обработанного кольца до разрезки на два полукольца 7.6.0.2 Каждую накладку 1 (2 шт.) (см. рисунок 6) следует крепить двумя шпильками для аппара ■ тов диаметром до 426мм и четырьмя шпильками - для аппаратов большего диаметра. 1Z"±3tf Рисуноя б — Размеры и оп;уленен^я размеров деталей плавающей гслоели 7.6.13 Предельное отклонение диаметров О, и D2 (см. рисунок 7) трубных решеток должно соответствовать 613поГОСТ25347. «Я Рисунок 7 — Узлы йлейинення мшел! ох и флеьцее

Если К. < 0. то укрепление кольцами жесткости не требуется. В диапазоне 0<К. <2^-1 расстояние между двумя кольцами жесткости вычисляют по формуле •(30) площадь поперечного сечения кольца жесткости вычисляют по формуле А,>/,(5-С)^-К4(«кФк Если к, >- 1 то толщину стенки необходимо увеличивать до такого размера, чтобы выполня лось следующее условие: 0 << 2^- - 1 Фр Примечание — При определении площади поперечного сечения кольца жесткости At следует учитывать прибавку с, для компенсации коррозии. 5.4.1.2 Допускаемое внутреннее избыточное давление вычисляют из условия [р] = min {[р],; {р]г}.(32) Допускаемое внутреннее избыточное давление, определяемое из условий прочности всей обечайки, вычисляют по формуле 2ЮЧ(з-с) + 2^[о1,Ф.(33) lPh "D * (s - с)' Допускаемое внутреннее избыточное давление, определяемое из условий прочности обечайки между двумя соседними кольцами жесткости, вычисляют по формуле tPh -- 24™ «Ь где 5.4.2 Обечайки с кольцами жесткости, нагруженные наружным давлением

А — ширина наплавки или сварного соединения со стороны трубной решетки, мм (см. рисунок 1); Л, — ширина полосы скольжс-иия. мм (см. рисунок 3/: А2 — высота полосы скопьжс-ния. мм (см. рисунок 3); В — глубина наплавки или выступа разделки кромки в трубной решетке, мм (см. рисунок 1). D - ■ наруж/ /ый диаметр аппарата, мм (см. таблицу 4); D1 —диаметр решетки, мм (см. рисунок 7); D-, - диаметр решетки, мм (см. рисунок 7); D3 -- наружный диаметр решетки мм (см. рисунок 6), Dd — внутренний диаметр полукольца, мм (см. рисунок 6). d0 — диаметр шпильки а зоне резьбы, мм (см. рисунок 6): П — высота соединительного выступа. мм (см. рисунок 7 к — высота ьлкллдки. мм (см. рисунок 6); К — катет углового шва. мм (см. рисунок 1); — катит углового шва. мы (см. рисунок ?}; Л — предельное отклонение габаритных и присоединительных размеров, мм (см. рисунок 5J; М — предельное отклонение торца фланца штуцера от вс-ртикальяои оси, мм (см. таблицу 1 fj; Рг — расчетное давление в кожухе, МПа (см. таблицу 14); Р-г - расчетное давлении в трубах, МПа (см. таблицу 14). R — радиус проточки, мм (см. рисунок 1): S— толщина колцовои обечайки кожуха, мм (см. рисунок 1); Sc — толщина стенки аппарата, мм (см. таблицу 4); S, - толщина соединительного выступа, мм (см. рисунок 1). Sri — толщина перегородки в центре, мм (см. рисунок 8);
Вопрос: Какова допустимая разность диаметрального зазора между кожухом и поперечной перегородкой для аппаратов типов Н и К, внутренний диаметр которых больше 1400 мм?
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
Ответ:
Допустимая разность диаметрального зазора между кожухом и поперечной перегородкой для аппаратов типов Н и К, внутренний диаметр которых больше 1400 мм, составляет не более 10 мм.


**<font color='magenta'>Total time:</font>** 8.422 sec.

### Evaluate on QA

In [11]:
import transformers

transformers.logging.set_verbosity_error()

In [32]:
from utils import find_hit

In [41]:
test_qa_df = pd.read_csv('data/qa_on_gost.csv')

In [44]:
test_qa_df.head()

,question,answer,context,akela_answer,reranked_paragraphs,hyde_doc
0,Какие физико-химические свойства характеризуют...,\nФизико-химические свойства конденсата включа...,компонентный состав и физико-химические свойст...,Физико-химические свойства конденсата включаю...,['физико-химические свойства газа в пластовых ...,"Конденсат, согласно ГОСТ 10688-2014 ""Конденсат..."
1,Какие физико-химические свойства пластовых вод...,\nВ документе описаны следующие физико-химичес...,компонентный состав и физико-химические свойст...,В документе описаны следующие физико-химическ...,['компонентный состав и физико-химические свой...,"В ГОСТ 12897-2019 ""Пластовые воды. Технические..."
2,Какие свойства пород-коллекторов и их взаимосв...,\nВ документе подробно описываются свойства по...,компонентный состав и физико-химические свойст...,В документе упоминаются следующие свойства по...,['компонентный состав и физико-химические свой...,"В соответствии с ГОСТом 12.2.007-2014 ""Породы-..."
3,Что служит критерием опасности коррозии для тр...,\nКритерием опасности коррозии для трубопровод...,"Примечания Для трубопроводов тепловых сетей, п...",Критерием опасности коррозии для трубопроводо...,"['Примечания Для трубопроводов тепловых сетей,...","В соответствии с ГОСТ 1629-2012 ""Тепловые сети..."
4,Как оценивается биокоррозия в соответствии с д...,\nВоздействие фактора биокоррозии оценивается ...,"Примечания Для трубопроводов тепловых сетей, п...",Биокоррозия оценивается на основании следующи...,"['Примечания Для трубопроводов тепловых сетей,...","В соответствии с ГОСТом ""Биокоррозия. Методы и..."


In [15]:
retrieval_chain = rag_pipeline.build_chain(retrieval_only=True)

In [22]:
retrieval_results = []

for q in tqdm(test_qa_df['question']):
    retrieval_results.append(retrieval_chain.invoke(q))

  0%|          | 0/397 [00:00<?, ?it/s]

In [43]:
test_qa_df['hyde_doc'] = [r['hyde_doc'] for r in retrieval_results]
test_qa_df.to_csv(f'data/test_qa_hyde_docs.csv', index=False)

In [34]:
test_qa_df['retrieved_docs'] = [[d.page_content for d in r['retrieved_docs']] for r in retrieval_results]

In [39]:
test_qa_df['hitrate'] = test_qa_df.apply(lambda x: find_hit(x['context'], x['retrieved_docs'], prefix_char=None), axis=1)

In [40]:
test_qa_df.notna().sum() / len(test_qa_df)

question               1.00000
answer                 1.00000
context                1.00000
akela_answer           1.00000
reranked_paragraphs    1.00000
retrieved_docs         1.00000
hyde_doc               1.00000
hitrate                0.20403
dtype: float64

## Synthetic questions

In [8]:
chunks = vectordb.get()

In [9]:
documents = []
for i in range(len(chunks['documents'])):
    page_content = chunks['documents'][i]
    chapter = chunks['metadatas'][i]['chapter']
    file_name = chunks['metadatas'][i]['file_name']
    source = chunks['metadatas'][i]['source']
    metadata = {
        'chapter': chapter,
        'file_name': file_name,
        'source': source
    }
    
    documents.append(Document(page_content=page_content, metadata=metadata))

In [10]:
generator = TestsetGenerator.from_langchain(
    generator_llm=llm,
    critic_llm=llm,
    embeddings=embeddings,
    docstore=vectordb
)

In [11]:
import transformers

transformers.logging.set_verbosity_error()

In [ ]:
testset = generator.generate_with_langchain_docs(documents, test_size=10, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})

In [ ]:
testset